In [0]:
python_list = [('Lio', 31, 50.0, 'Foot'), ('Lebron', 33, 38.5, 'Basket'),
        ('Novak', 31, 15.0, 'Tennis'), ('Kobe', 40, 24.5, 'Basket'),
        ('Cristiano', 34, 30.0, 'Foot'), ('Neymar',26 ,36.8, 'Foot')]

In [0]:
df = sqlContext.createDataFrame(python_list,['name','age','salary','sport'])
df.display()

name,age,salary,sport
Lio,31,50.0,Foot
Lebron,33,38.5,Basket
Novak,31,15.0,Tennis
Kobe,40,24.5,Basket
Cristiano,34,30.0,Foot
Neymar,26,36.8,Foot


In [0]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: double (nullable = true)
 |-- sport: string (nullable = true)



In [0]:
df.sort(df.salary,ascending=False).display()

name,age,salary,sport
Lio,31,50.0,Foot
Lebron,33,38.5,Basket
Neymar,26,36.8,Foot
Cristiano,34,30.0,Foot
Kobe,40,24.5,Basket
Novak,31,15.0,Tennis


In [0]:
#Display minimum, maximum and average salary for each sport

df.groupby(df.sport).max("salary").show()

+------+-----------+
| sport|max(salary)|
+------+-----------+
|  Foot|       50.0|
|Basket|       38.5|
|Tennis|       15.0|
+------+-----------+



In [0]:
df.groupby(df.sport).avg("salary").show()

+------+-----------------+
| sport|      avg(salary)|
+------+-----------------+
|  Foot|38.93333333333333|
|Basket|             31.5|
|Tennis|             15.0|
+------+-----------------+



In [0]:
df.groupby(df.sport).min("salary").show()

+------+-----------+
| sport|min(salary)|
+------+-----------+
|  Foot|       30.0|
|Basket|       24.5|
|Tennis|       15.0|
+------+-----------+



In [0]:
df.createOrReplaceTempView("sport")

In [0]:
spark.sql("select * from sport").show()

+---------+---+------+------+
|     name|age|salary| sport|
+---------+---+------+------+
|      Lio| 31|  50.0|  Foot|
|   Lebron| 33|  38.5|Basket|
|    Novak| 31|  15.0|Tennis|
|     Kobe| 40|  24.5|Basket|
|Cristiano| 34|  30.0|  Foot|
|   Neymar| 26|  36.8|  Foot|
+---------+---+------+------+



In [0]:
#moyen salary 
spark.sql("select Sport, avg(salary) as moySal from sport group by sport order by moySal desc").show()

+------+-----------------+
| Sport|           moySal|
+------+-----------------+
|  Foot|38.93333333333333|
|Basket|             31.5|
|Tennis|             15.0|
+------+-----------------+



# TP1 MISSING DATA

In [0]:
#Load the missing-data CSV dataset into Spark cluster (using SparkSession)
missingDF = spark.read.csv("/FileStore/tables/missingData.csv",inferSchema =True,header=True)
#infershama -> on dit qu'on a plusieurs types de données
missingDF.cache()

Out[1]: DataFrame[Id: string, Name: string, Salary: double, Sport: string]

In [0]:
missingDF

Out[7]: DataFrame[Id: string, Name: string, Salary: double, Sport: string]

In [0]:
#Display and explore the loaded DataFrame (use show(), printSchema(), describe())
missingDF.show()
missingDF.printSchema()
missingDF.describe()

+----+----------+------+------+
|  Id|      Name|Salary| Sport|
+----+----------+------+------+
| id1|       Leo|  null|  foot|
| id2|      null|  null|  null|
| id3|      null| 445.0|  foot|
| id4|Christiano| 566.0|  foot|
| id5|    Lebron| 456.0|basket|
| id6|     Roger|  null|tennis|
| id7|      null| 600.0|  foot|
| id8|      Rafa| 300.0|tennis|
| id9|     Novak| 400.0|tennis|
|id10|       ney| 500.0|  foot|
+----+----------+------+------+

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Salary: double (nullable = true)
 |-- Sport: string (nullable = true)

Out[3]: DataFrame[summary: string, Id: string, Name: string, Salary: string, Sport: string]

In [0]:
missingDF2 = missingDF.na.drop()

In [0]:
missingDF2.show()

+----+----------+------+------+
|  Id|      Name|Salary| Sport|
+----+----------+------+------+
| id4|Christiano| 566.0|  foot|
| id5|    Lebron| 456.0|basket|
| id8|      Rafa| 300.0|tennis|
| id9|     Novak| 400.0|tennis|
|id10|       ney| 500.0|  foot|
+----+----------+------+------+



In [0]:
#Display the average salary per sport

#I remove the null value because they change the result

missingDF2.groupBy(missingDF2.Sport).avg("Salary").show()

+------+-----------+
| Sport|avg(Salary)|
+------+-----------+
|  foot|      533.0|
|basket|      456.0|
|tennis|      350.0|
+------+-----------+



In [0]:
#Display, in one table, the minimum and the maximum salary per sport

missingDF.createOrReplaceTempView("Sport2")
#moyen salary 
spark.sql("select Sport, min(salary),max(Salary) from sport2 group by sport").show()

+------+-----------+-----------+
| Sport|min(salary)|max(Salary)|
+------+-----------+-----------+
|  null|       null|       null|
|  foot|      445.0|      600.0|
|basket|      456.0|      456.0|
|tennis|      300.0|      400.0|
+------+-----------+-----------+



In [0]:
#Create a new DataFrame1 without missing values (use drop())

DataFrame1 = missingDF.na.drop()

In [0]:
#Create a new DataFrame2 with at least 2 non null values per row

DataFrame2 = 